## Exploratory Analysis for finding relevant factors and Forming dictionaries using Lexicons

### Objective: Extracting Patterns to use to extract relevant Risk Factors

##### Note: Still under progress below... Hence not fully documented and commented

##### Categories to classify based on the following link: https://afsp.org/about-suicide/risk-factors-and-warning-signs/

In [9]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict, Counter
import pandas as pd

import nltk
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from collections import Counter
import math, re
from itertools import zip_longest
from datetime import datetime
from functools import reduce

import sklearn
print(sklearn.__version__)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics
from sklearn import model_selection
from sklearn.externals import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

import re

from afinn import Afinn
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sklearn_crfsuite


0.18.1


C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Uses of Various Lists in this code
#### List a for storing number of sentences of each comments
#### List b for storing storing pos tag of each word in the sentences
#### List c for storing the comment id in the order of occurrence
#### List d for storing the pos tag of sentences as sentence format itself
#### List helper is used for recording the sentence, the comment it is related to and the index of the sentence as in the order of being read
#### List sents contains the the tokenized sentences of each comment in one iteration
#### List text contains the words tokenized by nltk per comment
#### List all_sents contains all the sentences that have been tokenized by nltk
#### List pos_tagged_sent_without_BK contains all the pos_tagged sentences in d but without 'BK'
#### List comments contains all the comments alone
#### all_sent_without_BK contains all the sentences in all_sents but without 'BK'

### Note: BK is in the CSV file to preserve the Line Breaks which occur in comments

In [6]:
a = []
b = []
c = []
d = []
helper = []
csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Top_Level_Comment_Without_Removed.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
i=0
for row in reader:
    if i==0:
        i=i+1
        continue
    sents = sent_tokenize(row[2])
    for one_sent in sents:            
        helper.append([one_sent, row[0]])
    d = d + sents
    text = nltk.word_tokenize(row[2])
    a.append(len(sents))
    b = b + nltk.pos_tag(text)
    c.append(row[0])
all_sents = d
d = [nltk.word_tokenize(sent) for sent in d]
d = [nltk.pos_tag(sent) for sent in d]
csvfile.close()

pos_tagged_sent_without_BK = list()
for lister in d:
    pos_tagged_sent_without_BK.append([t for t in lister if not '[' in t[0] and not 'BK' in t[0] and not ']' in t[0]  ])

comments = list()
csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Top_Level_Comment_Without_Removed.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
i=0
for row in reader:
    if i==0:
        i=i+1
        continue
    comments.append(row[2])
csvfile.close()

all_sent_without_BK = list()
for sent in all_sents:
    all_sent_without_BK.append(sent.replace("[BK]", ""))

counter = 0
for row in helper:
    row.append(counter)
    counter+=1

### Experimenting with Different Lexicons and Getting Sentiment Scores for the Sentences

### Using Afinn Lexicon

In [6]:
afinn = Afinn(emoticons=True)
Afinn_score_sentences = [(afinn.score(sent), afinn.find_all(sent), sent) for sent in all_sent_without_BK]

In [44]:
Afinn_emotions = Afinn_score_sentences

all_words = list()
all_words = [afinn.find_all(sent) for sent in all_sent_without_BK]

all_words = [item for sublist in all_words for item in sublist]

senti_Afinn = list()
for sentence in all_sent_without_BK:
    senti_Afinn.append(afinn.score(sentence))

In [8]:
emotion_df = pd.DataFrame(Afinn_emotions)
emotion_df.columns= ['Sentiment Score', 'Words Considered','Sentence']
emotion_df.head()

,Sentiment Score,Words Considered,Sentence
0,-3.0,"[friendly, suicide, difficult, scary]","Hey everyone,This is just a friendly reminder ..."
1,4.0,"[feeling, like, please]","If you are feeling like you are not coping, pl..."
2,-1.0,"[crisis, ://, support]",There are [crisis services worldwide](https://...
3,-1.0,"[unlikely, problems, help]",These centres are designed to give temporary r...
4,1.0,"[like, suffering, illness, hard, recommend, ap...","Otherwise, if you feel like you may be sufferi..."


### Using Bing Liu's Lexicon

In [13]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [10]:
negList = []
posList = []
wordDict = defaultdict(list)

with open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/BingLiu/positive-words.txt', 'r') as f:
    reader = csv.reader(f)
    headerRows = [i for i in range(0, 35)]
    for row in headerRows:
        next(reader)
    for word in reader:
        posList.append(word[0])
        wordDict[word[0]].append('positive')

with open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/BingLiu/negative-words.txt', 'r') as f:
    reader = csv.reader(f)
    headerRows = [i for i in range(0, 35)]
    for row in headerRows:
        next(reader)
    for word in reader:
        negList.append(word[0])
        wordDict[word[0]].append('negative')

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
sentence = list()
emotionCounts = list()
Bing_neg = list()
i = 0
for sent in pos_tagged_sent_without_BK:
    emoCount = Counter()
    wordColl = list()
    for word in sent:
        p = wordnet_lemmatizer.lemmatize(word[0],get_wordnet_pos(word[1]))
        emoCount += Counter(wordDict[p.lower()])
        if wordDict[p.lower()] != []:
            wordColl.append(word[0]) 
        if wordDict[p.lower()] == ['negative']:
            Bing_neg.append(word[0])
    emotionCounts += [(emoCount['positive']-emoCount['negative'], wordColl, all_sent_without_BK[i])]
    i += 1

In [11]:
BingLiu_emotions = emotionCounts

In [12]:
emotion_df = pd.DataFrame(BingLiu_emotions)
emotion_df.columns= ['Sentiment Score', 'Words Considered','Sentence']
emotion_df.head()

,Sentiment Score,Words Considered,Sentence
0,-1,"[friendly, suicide, difficult, tough, scary]","Hey everyone,This is just a friendly reminder ..."
1,2,"[like, available]","If you are feeling like you are not coping, pl..."
2,0,"[crisis, support]",There are [crisis services worldwide](https://...
3,-1,"[relief, overwhelming, unlikely, problems, tough]",These centres are designed to give temporary r...
4,0,"[like, suffering, illness, hard, recommend, ap...","Otherwise, if you feel like you may be sufferi..."


### Using Senti WordNet

In [13]:
def get_SentiWordNet_Score():
    i = 0
    SentiWord = list()
    get_SentiWordNet_Score.SentiWordnet_neg = list()
    for sent in pos_tagged_sent_without_BK:
        sentiment = 0.0
        tokens_count = 0
        ObjScore = 0.0
        used_words = list()
        for word,tag in sent:
            wn_tag = get_wordnet_pos(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = wordnet_lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            if swn_synset.pos_score() - swn_synset.neg_score() < 0:
                get_SentiWordNet_Score.SentiWordnet_neg.append(word)
            tokens_count += 1
            ObjScore = ObjScore + 1 - (swn_synset.pos_score() + swn_synset.neg_score())
            ObjScore /= tokens_count
            used_words.append(word)
        SentiWord += [(sentiment, ObjScore, used_words, all_sent_without_BK[i])]
        i += 1
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return None
 
    # sum greater than 0 => positive sentiment
    return SentiWord

In [14]:
SentiWord_emotions = get_SentiWordNet_Score()

In [17]:
Sentinet_neg = get_SentiWordNet_Score.SentiWordnet_neg # To get list of negative words as categrized by SentiWordnet

In [15]:
emotion_df = pd.DataFrame(SentiWord_emotions)
emotion_df.columns= ['Sentiment Score','Avg Subjective/Objective Score', 'Words Considered','Sentence']
emotion_df.head()

,Sentiment Score,Avg Subjective/Objective Score,Words Considered,Sentence
0,-1.500,0.045301,"[just, a, friendly, reminder, discussion, suic...","Hey everyone,This is just a friendly reminder ..."
1,-0.125,0.119692,"[like, not, there, resources, out, there, avai...","If you are feeling like you are not coping, pl..."
2,-0.125,0.278125,"[There, crisis, services, https, support]",There are [crisis services worldwide](https://...
3,-1.500,0.032887,"[centres, temporary, relief, feelings, while, ...",These centres are designed to give temporary r...
4,-0.500,0.071828,"[like, may, a, mental, illness, or, it, hard, ...","Otherwise, if you feel like you may be sufferi..."


### Using TextBlob

In [16]:
#[-1.0, 1.0] for positive and negative sentiments. 
#The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
TextBlob_emotions = list()
for sent in all_sent_without_BK:
    blob = TextBlob(sent)
    blob.sentences
    for sentence in blob.sentences:
        TextBlob_emotions += [(sentence.sentiment.polarity, sentence.sentiment.subjectivity, sent)]

In [17]:
emotion_df = pd.DataFrame(TextBlob_emotions)
emotion_df.columns= ['Sentiment Score', 'Subjective/Objective Score','Sentence']
emotion_df.head()

,Sentiment Score,Subjective/Objective Score,Sentence
0,-0.253472,0.833333,"Hey everyone,This is just a friendly reminder ..."
1,0.400000,0.400000,"If you are feeling like you are not coping, pl..."
2,0.000000,0.000000,There are [crisis services worldwide](https://...
3,-0.129630,0.777778,These centres are designed to give temporary r...
4,0.128333,0.415000,"Otherwise, if you feel like you may be sufferi..."


### Using Vader Lexicon

In [18]:
#positive sentiment: compound score >= 0.5
#neutral sentiment: (compound score > -0.5) and (compound score < 0.5)
#negative sentiment: compound score <= -0.5
Vader_emotions = list()
senti_Vader = list()
sid = SentimentIntensityAnalyzer()
for sentence in all_sent_without_BK:
    ss = sid.polarity_scores(sentence)
    senti_Vader.append(ss['compound'])
    Vader_emotions += [(ss['compound'], sentence)]

C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [19]:
# Normalizing the scores of Vader Lexicon between [-5,+5]

for i, value in enumerate(senti_Vader):
    if value<0.5 and value> -0.5:
        senti_Vader[i] = 0
    elif value>= 0.5 and value<0.6:
        senti_Vader[i] = 1
    elif value>= 0.6 and value<0.7:
        senti_Vader[i] = 2
    elif value>= 0.7 and value<0.8:
        senti_Vader[i] = 3
    elif value>= 0.8 and value<0.9:
        senti_Vader[i] = 4
    elif value >= 0.9 and value < 1:
        senti_Vader[i] = 5
    elif value <= -0.5 and value > -0.6:
        senti_Vader[i] = -1
    elif value <= -0.6 and value> -0.7:
        senti_Vader[i] = -2
    elif value <= -0.7 and value> -0.8:
        senti_Vader[i] = -3
    elif value <= -0.8 and value> -0.9:
        senti_Vader[i] = -4
    elif value <= -0.9 and value> -1:
        senti_Vader[i] = -5
    

In [20]:
emotion_df = pd.DataFrame(Vader_emotions)
emotion_df.columns= ['Sentiment Score','Sentence']
emotion_df.head()

,Sentiment Score,Sentence
0,-0.8313,"Hey everyone,This is just a friendly reminder ..."
1,0.2589,"If you are feeling like you are not coping, pl..."
2,0.4019,There are [crisis services worldwide](https://...
3,0.3818,These centres are designed to give temporary r...
4,-0.0258,"Otherwise, if you feel like you may be sufferi..."


### Using NRC Lexicon

In [21]:
wordList = defaultdict(list)
emotionList = defaultdict(list)
with open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/NRC/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for word, emotion, present in reader:
        if int(present) == 1:
            #print(word)
            wordList[word].append(emotion)
            emotionList[emotion].append(word)

In [22]:
wordnet_lemmatizer = WordNetLemmatizer()
sentence = list()
NRC_neg = list()
NRC_emotion = list()
i = 0
for sent in pos_tagged_sent_without_BK:
    emoCount = Counter()
    wordColl = list()
    for word in sent:
        p = wordnet_lemmatizer.lemmatize(word[0],get_wordnet_pos(word[1]))
        emoCount += Counter(wordList[p.lower()])
        if 'negative' in wordList[p.lower()]:
            NRC_neg.append(word[0])
        if wordDict[p.lower()] != []:
            wordColl.append(word[0]) 
    NRC_emotion  += [(emoCount['anger'], emoCount['anticipation'],emoCount['disgust'], emoCount['fear'], emoCount['joy'],
                       emoCount['sadness'], emoCount['surprise'], emoCount['trust'], emoCount['positive'], 
                       emoCount['negative'], wordColl, all_sent_without_BK[i])]
    i += 1

In [23]:
emotion_df = pd.DataFrame(NRC_emotion)
emotion_df.columns= ['Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust','Poitive','Negative','Words Used','Sentence']
emotion_df.head()

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Poitive,Negative,Words Used,Sentence
0,2,3,1,3,2,3,1,2,3,3,"[friendly, suicide, difficult, tough, scary]","Hey everyone,This is just a friendly reminder ..."
1,0,0,0,1,0,0,0,1,0,0,"[like, available]","If you are feeling like you are not coping, pl..."
2,0,0,0,0,0,0,0,1,1,1,"[crisis, support]",There are [crisis services worldwide](https://...
3,1,1,1,2,1,3,1,1,2,4,"[relief, overwhelming, unlikely, problems, tough]",These centres are designed to give temporary r...
4,0,0,0,1,0,1,0,2,3,2,"[like, suffering, illness, hard, recommend, ap...","Otherwise, if you feel like you may be sufferi..."


### MPQA Lexicon

In [24]:
MPQA_FILE = "C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/MPQA_Lexicon/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff"
match_count = 0
add_count = 0
word_MPQA = defaultdict(list)
with open(MPQA_FILE) as f:
    lines = f.read().splitlines()
    for line in lines:
        pairs = line.split(" ")
        types = pairs[0].split("=")
        word = pairs[2].split("=")
        pos = pairs[3].split("=")
        polar = pairs[5].split("=")
        if pos[1] == 'noun':
            pos[1] = wordnet.NOUN
        elif pos[1] == 'verb':
            pos[1] = wordnet.VERB
        elif pos[1] == 'adj':
            pos[1] = wordnet.ADJ
        elif pos[1] == 'anypos':
            pos[1] = wordnet.NOUN
        word_MPQA[word[1]].append((types[1], pos[1], polar[1]))

In [25]:
wordnet_lemmatizer = WordNetLemmatizer()
sentence = list()
MPQA_emotion = list()
i = 0
MPQA_neg= list()
for sent in pos_tagged_sent_without_BK:
    emoCount = Counter()
    wordColl = list()
    strong_subj_word = list()
    for word in sent:
        pos = get_wordnet_pos(word[1])
        p = wordnet_lemmatizer.lemmatize(word[0], pos)
        if word_MPQA[p.lower()] != []:
            wordColl.append(word[0])
            for tup in word_MPQA[p.lower()]:
                if pos == tup[1]:
                    subj = tup[0]
                    polar = tup[2]
                    emoCount += Counter([polar])
                    if subj == 'strongsubj':
                        strong_subj_word.append(word[0])
                        if polar == 'negative':
                            MPQA_neg.append(word[0])
                else:
                    continue
    MPQA_emotion  += [(emoCount['positive'] - emoCount['negative'], emoCount['neutral'], strong_subj_word, wordColl, all_sent_without_BK[i])]
    i += 1

In [26]:
emotion_df = pd.DataFrame(MPQA_emotion)
emotion_df.columns= ['Sentiment Score','No. of Neutral Words','Strong Subjective Words','Words Considered','Sentence']
emotion_df.head()

,Sentiment Score,No. of Neutral Words,Strong Subjective Words,Words Considered,Sentence
0,-2,1,"[friendly, scary, feelings]","[just, friendly, suicide, really, difficult, s...","Hey everyone,This is just a friendly reminder ..."
1,1,1,"[please, know]","[feeling, like, please, know]","If you are feeling like you are not coping, pl..."
2,0,0,[support],"[crisis, support]",There are [crisis services worldwide](https://...
3,0,1,"[relief, feelings]","[relief, feelings, overwhelming, unlikely, pro...",These centres are designed to give temporary r...
4,-2,0,[recommend],"[feel, like, suffering, illness, hard, recomme...","Otherwise, if you feel like you may be sufferi..."


#### Reading Required Dictionaries

In [4]:
ability_verbs = list()
FPRP = list()
WPRP = list()
SPRP = list()
swear = list()
POSEmo = list()
intensifier = list()
negative = list()
positive = list()
negations = list()
methods = list()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Final_Methods.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    methods.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/abilityverbs.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    ability_verbs.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/FPRP.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    FPRP.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/WPRP.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    WPRP.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/SPRP.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    SPRP.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/All_Negative_Words.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    negative.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/Negations.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    negations.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/swear.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    swear.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/POSEmo.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    POSEmo.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/extreme_intensifiers_level0.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    intensifier.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/extreme_intensifiers_level1.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    intensifier.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/All_Postive_Words.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    positive.append(row[0])
csvfile.close()

methods = [w.replace(w, w.lower()) for w in methods]
ability_verbs = [w.replace(w, w.lower()) for w in ability_verbs]
FPRP = [w.replace(w, w.lower()) for w in FPRP]
WPRP = [w.replace(w, w.lower()) for w in WPRP]
SPRP = [w.replace(w, w.lower()) for w in SPRP]
swear = [w.replace(w, w.lower()) for w in swear]
POSEmo = [w.replace(w, w.lower()) for w in POSEmo]
intensifier = [w.replace(w, w.lower()) for w in intensifier]
negative = [w.replace(w, w.lower()) for w in negative]
positive = [w.replace(w, w.lower()) for w in positive]
negations = [w.replace(w, w.lower()) for w in negations]

In [5]:
set(negative).intersection(intensifier)

{'awfully',
 'drastically',
 'dreadfully',
 'enormously',
 'excessively',
 'exorbitantly',
 'extravagantly',
 'extremely',
 'extremism',
 'fastidiously',
 'hardly',
 'horrendously',
 'horribly',
 'horrifyingly',
 'immoderately',
 'impossibly',
 'inordinately',
 'insanely',
 'insignificantly',
 'loathsomely',
 'radically',
 'seriously',
 'severely',
 'sorely',
 'terribly',
 'unreasonably',
 'unusually',
 'utterly',
 'violently',
 'wildly'}

### Finding only the sentences with negative sentiment score by weighting the significant lexicons

In [27]:
i = 0
neg_sent = list()
for sent1, sent2, sent3, sent4, sent5, sent6 in zip(Afinn_emotions, BingLiu_emotions, SentiWord_emotions, TextBlob_emotions, Vader_emotions, MPQA_emotion):
    count = 0
    flag = 0
    if sent1[0] < 0: #Afinn
        count += 1
    if sent2[0] < 0: #Bing Liu
        count += 0.5
    if sent3[0] < 0: #SentiWord
        count += 0.5
    if sent4[0] < 0: #TextBlob
        count += 1
    if sent5[0] <= 0.5: #Vader
        count += 1.5
    if sent6[0] < 0: #MPQA
        count += 0.5
    if count >= 3.5 :
        neg_sent.append(sent1[2])
print(len(neg_sent))

4327


### Reverse Engineering POS Tags by getting Negative Words from Sentences and Getting the words before till the Personal Pronoun

##### Note: Can be changed accordingly to detect our tags

In [14]:
sentence = list()
all_sentences = list()
wordnet_lemmatizer = WordNetLemmatizer()
for count, sent in enumerate(pos_tagged_sent_without_BK):
    for i, word in enumerate(sent) :
        p = wordnet_lemmatizer.lemmatize(word[0], get_wordnet_pos(word[1]))
        if p.lower() in negative :
            j = i
            sen = sent[j]
            flag = 0
            sente = list()
            senti = list()
            while j >= 0 and i >= 0 and sen is not None : 
                sente.append(sen)
                senti = [sen[1]] + senti
                if  sen[1].startswith("PRP") or sen[1].startswith("WP") :
                    #print("FOUND PRONOUN {}".format(count))
                    flag = 1
                    break
                j -= 1
                sen = sent[j]
                #if j < 0:
                    #print("Didn't find a pronoun in sentence {}".format(count))
            if flag == 1:
                sentence.append(senti)
                all_sentences.append(sente)
            break

all_sentences_edit = [list(reversed(sent)) for sent in all_sentences]

In [23]:
def pos_seq_to_sent_seq(tags, pos_seq):
    tag_order = list()
    i = 0
    for sentence in pos_seq:
        tag_order = []
        for element in sentence:
            tag_order = tag_order + [element[1]]
        if tags == tag_order:
            for element in sentence:
                print(element[0], end=" ")
            i += 1
            print()

In [34]:
tags = ['PRP', 'VBD', 'JJ']
pos_seq_to_sent_seq(tags, all_sentences_edit)

It was scary 
It was despicable 
I got insane 
it was crazy 
It was bizarre 
I was bitter 
I was livid 
I felt sad 
I felt guilty 
I was sad 
I was desperate 
I was furious 
I was ravenous 
me was resentful 
I was furious 
he was angry 
I was dizzy 
It was melodramatic 
me was enormous 
It was bumpy 
I was sad 
I was sick 
It was sick 
I was dead 
me had severe 
I was unable 
I worked hard 
I was sad 
I was angry 
I was unconscious 
I was dead 
I was mortal 
I felt bad 
I was dead 
I was unconscious 
It was uncomfortable 
I was dead 
I was numb 
I attempted suicide 
I was unsuccessful 
He seemed upset 
I felt sick 
I felt sick 
he was wrong 
I was vile 
It was hurtful 
I was delirious 
I felt bad 
It was weird 
I felt useless 
I was wrong 
I was allergic 
I felt ashamed 
it was inconvenient 
it was horrible 
I was unconscious 
It was weird 
I felt enormous 
I was afraid 
I had vague 
It was strange 
I felt awful 
I felt selfish 
It was strange 
I were dead 
you were unsuccessful 
I was

In [36]:
n = 3
count = 0
for (w1,t1), (w2,t2), (w3,t3) in nltk.ngrams(b, n):
    if (t1 =='PRP'  and t2 == 'VBD' and t3== 'VBN'): #and wordnet_lemmatizer.lemmatize(w3.lower(), get_wordnet_pos(nltk.pos_tag([w3.lower()])[0][1])) in negative):
        print(w1, w2, w3) 

I felt embarrassed
I had attempted
I had labeled
I had done
I was put
I was loaded
I was deployed
I was treated
I had broken
I had been
I was terrified
I was crushed
I were supposed
I had changed
I was scared
I was taken
I was given
I was discovered
I had tried
I had been
I had decided
I got tired
I was kept
I had succeeded
I was relieved
I was handcuffed
I had been
I had taken
she had found
I got kicked
I had squeezed
I tied managed
it had happened
I had intended
I got sent
it had been
I was bullied
he was scared
I was told
I had taken
I was stitched
I was admitted
I was bullied
I had taken
I had started
it was directed
I had left
I had done
I was put
I was given
I stayed curled
I was given
I was caught
I had cut
I was interviewed
I was embarrassed
me had been
he was depressed
I had worn
They had stripped
I had gotten
it had worked
it was assumed
I was given
I had pneumonia
it had been
I got consumed
I had tried
I had ended
I was bullied
I was gone
I had been
I had accepted
I had gone

762

### Making a Map For the Risk Factor Categorization

In [37]:
class Tree(object):
    "Generic tree node."
    def __init__(self, name='root', children=None):
        self.name = name
        self.children = []
        self.parent = []
        if children is not None:
            for child in children:
                assert isinstance(child, Tree)
                self.add_child(child)
                child.parent.append(self)            
    def __repr__(self):
        return self.name
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)
    def add_parent(self, node):
        assert isinstance(node, Tree)
        self.parent.append(node)
    def get_child(self):
        return self.children
    def get_parent(self):
        return self.parent
def find_node(root, name):
    if root.name == name:
        return root
    for child in root.children:
        child = find_node(child, name)
        if child is not None:
            return child
    return None

In [38]:
root = Tree('Root', [Tree('Health Factors', [Tree('Depression'), Tree('Schizophrenia'), Tree('Personality Disorder'), 
                                             Tree('Conduct Disorder'), Tree('Psychotic Disorder'), Tree('Anxiety Disorder')]),
               Tree('Environmental Factors', [Tree('Stressful Life Events'), Tree('Prolonged Stress Factors'), Tree('Personality Disorder'),
                                              Tree('Lethal Means Access'), Tree('Inspired Suicides')]),
               Tree('Historical Factors', [Tree('Previous Suicide Attempts'),Tree('Family History')]),
               Tree('Other Factors')])

In [39]:
def find_path(root, start, path=[]):
    path = path + [start]
    if start == 'Root':
        return None
    if start in ('Health Factors','Environmental Factors','Historical Factors','Other Factors') :
        return path
    node = find_node(root, start) 
    if node is not None:
        while node.name not in ('Health Factors','Environmental Factors','Historical Factors','Other Factors') and node is not None:
            node = node.parent[0]
            path = path + [node.name]
    if node is not None and node.name in ('Health Factors','Environmental Factors','Historical Factors','Other Factors') :
        return path
    return None

In [40]:
path = find_path(root, 'Depression')
if path is None:
    print("No Path Exists")
else:
    for item in path:
        if item == path[-1]:
            print(item)
        else:
            print(item,"-->", end=" ")

Depression --> Health Factors


### Feature Extraction for CRF

In [17]:
# Takes time to execute... But used only once in the start... So use only once during first execution and comment it again
"""pos_tagged= list()
for sent in all_sent_without_BK:
    pos_tagged.append(nltk.pos_tag(nltk.word_tokenize(sent)))"""

In [45]:
scored_sentences = list()
for i, sent in enumerate(pos_tagged):
    NEG_word = list()
    cfprp = cwprp = csprp = cintfr = cpositive = cnegative = cablt = cswr = cneg = cnegintfr = flag = 0
    for j, word in enumerate(sent):
        if word[0].lower() in FPRP:
            cfprp += 1
        if word[0].lower() in WPRP:
            cwprp += 1
        elif word[0].lower() in SPRP:
            csprp += 1
        elif word[0].lower() in negations:
            cneg += 1
        elif word[0].lower() in ability_verbs:
            cablt += 1
        elif word[0].lower() in swear:
            cswr += 1
        if word[0].lower() in intensifier and word[0].lower() in negative:
            cnegintfr += 1
            NEG_word.append(word[0].lower())
        elif word[0].lower() in intensifier:
            cintfr += 1
        elif word[0].lower() in negative:
            cnegative += 1
            NEG_word.append(word[0].lower())
        elif word[0].lower() in positive:
            cpositive += 1
    for expr in POSEmo:
        matchObj = re.compile(r"%s" % expr)
        match = matchObj.findall(all_sent_without_BK[i])
        if (len(match))>0:
            flag = 1
            break
    scored_sentences.append([all_sent_without_BK[i], cfprp, csprp, cneg, cablt, cswr, cintfr, cnegative, cnegintfr, flag, cpositive, cwprp, senti_Vader[i], int(senti_Afinn[i])])

#### For getting our kind of Tags

In [15]:
def get_the_tag(word):
        tupled = tuple()
        if word[0].lower() in negations:
            tupled = (word[0], 'NEG')
            get_the_tag.flag = 1
        # word[1].startswith('PRP') and word[0].lower() in SPRP:
        elif word[0].lower() in SPRP:
            tupled = (word[0], 'SPRP')
        # word[1].startswith('PRP') and word[0].lower() in FPRP
        elif word[0].lower() in FPRP:
            tupled = (word[0], 'FPRP')
            get_the_tag.count_PRP += 1
        elif word[0].lower() in WPRP:
            tupled = (word[0], 'WPRP')
        elif word[0].lower() in ability_verbs:
            tupled = (word[0], 'ABLT')
        elif word[0].lower() in swear:
            tupled = (word[0], 'SWR')
        if word[0].lower() in positive:
            tupled = (word[0], 'POSITIVE')
        if word[0].lower() in intensifier and word[0].lower() in negative:
            tupled = (word[0], 'NEGINTSFR')
            get_the_tag.count_neg += 1
        elif word[0].lower() in intensifier:
            tupled = (word[0], 'INTSFR')
        elif word[0].lower() in negative:
            tupled = (word[0], 'NEGATIVE')
            get_the_tag.count_neg += 1
        for expr in POSEmo:
            matchObj = re.compile(r"%s" % expr)
            match = matchObj.findall(word[0].lower())
            if (len(match))>0:
                tupled = (word[0], 'POSEMO')
                break
        if len(tupled) == 0:
            tupled = (word[0], word[1])
        return tupled

In [18]:
ourtag = list()
count  = 0
for i, sent in enumerate(pos_tagged):
    
    tagger = list()
    get_the_tag.flag = 0
    get_the_tag.count_PRP = 0
    get_the_tag.count_neg = 0
    
    for j, word in enumerate(sent):
        tupled = tuple()
        tupled = get_the_tag(word)
        # For appending the NEG to signify words coming after a Negation
        #if get_the_tag.flag == 1:
        #    get_the_tag.flag += 1
        #elif get_the_tag.flag == 2:
        #    tupled = (tupled[0], tupled[1] + "_NEG")
        
        tagger.append(tupled)
    ourtag.append(tagger)
    count += 1

### Counts the number of occurrences of each sequence extracted

In [20]:
# List ourtagger consists of normal pos_tagged_sequences except for the words in our dictionaries which have the tags 
# that we have specified. The list starts with FPRP and ends with a NEGATIVE
ourtagger = list()
count  = 0
for i, sent in enumerate(all_sentences_edit):
    
    tagger = list()
    get_the_tag.flag = 0
    get_the_tag.count_PRP = 0
    get_the_tag.count_neg = 0
    
    for j, word in enumerate(sent):
        tupled = tuple()
        tupled = get_the_tag(word)
        # For appending the NEG to signify words coming after a Negation
        #if get_the_tag.flag == 1:
        #    get_the_tag.flag += 1
        #elif get_the_tag.flag == 2:
        #    tupled = (tupled[0], tupled[1] + "_NEG")
        
        tagger.append(tupled)
    ourtagger.append(tagger)

In [21]:
ourtagger_postag = list()
for sent in ourtagger:
    sentence = [l[1] for l in sent]
    ourtagger_postag.append(sentence)

nested_lst_of_tuples = [tuple(l) for l in ourtagger_postag]
count = Counter(elem for elem in nested_lst_of_tuples)
sorted(count.items(), key=lambda x: x[1], reverse = True)

[(('FPRP', 'NEGATIVE'), 505),
 (('FPRP', 'VBD'), 334),
 (('FPRP', 'VBD', 'NEGATIVE'), 214),
 (('SPRP', 'NEGATIVE'), 117),
 (('FPRP', 'VBD', 'VBN'), 104),
 (('FPRP', 'VBD', 'TO', 'NEGATIVE'), 100),
 (('SPRP', 'VBD'), 91),
 (('FPRP', 'VBD', 'DT', 'NEGATIVE'), 81),
 (('FPRP', 'VBD', 'INTSFR', 'NEGATIVE'), 79),
 (('FPRP', 'VBD', 'NEG', 'NEGATIVE'), 74),
 (('FPRP', 'VBD', 'VBG'), 71),
 (('FPRP', 'VBP', 'NEGATIVE'), 61),
 (('SPRP', 'VBD', 'NEGATIVE'), 55),
 (('FPRP', 'RB', 'NEGATIVE'), 53),
 (('FPRP', 'VBP', 'NEG', 'NEGATIVE'), 49),
 (('FPRP', 'INTSFR', 'VBD'), 44),
 (('FPRP', 'MD', 'NEGATIVE'), 42),
 (('FPRP', 'VBP', 'DT', 'NEGATIVE'), 41),
 (('SPRP', 'VBD', 'DT', 'NEGATIVE'), 40),
 (('FPRP', 'INTSFR', 'NEGATIVE'), 39),
 (('FPRP', 'VBP', 'VBN'), 30),
 (('WP', 'DT', 'NEGATIVE'), 29),
 (('FPRP', 'VBP', 'VBG'), 29),
 (('FPRP', 'JJ', 'NEGATIVE'), 29),
 (('SPRP', 'VBZ'), 26),
 (('FPRP', 'NN', 'NEGATIVE'), 26),
 (('SPRP', 'VBD', 'INTSFR', 'NEGATIVE'), 24),
 (('FPRP', 'NNS'), 23),
 (('FPRP', 'POSI

In [25]:
tags = ['FPRP', 'VBD', 'INTSFR', 'NEGATIVE']
pos_seq_to_sent_seq(tags, ourtagger)

I felt so overwhelmed 
I was really unsure 
I was just numb 
I was so lonely 
I was so stuck 
I was so nervous 
I was very delirious 
I felt more alone 
I was incredibly hazy 
I was so angry 
I was so upset 
I felt really alone 
I felt so weak 
I was so mad 
I was completely empty 
I was just sad 
I became more depressed 
I felt incredibly stupid 
I was so sorry 
I was really drunk 
I got really drunk 
I was completely alone 
I was completely helpless 
I had very little 
I was very suicidal 
I was so afraid 
I was so black 
I was super mad 
I felt really guilty 
I felt so sick 
I went so high 
me was really stupid 
I cared so little 
I was really bad 
I was just cold 
I was just alone 
I was so devoid 
I was really disappointed 
I was just angry 
I was honestly mad 
I was so upset 
I felt really high 
I was deeply ashamed 
I was incredibly upset 
I was really apathetic 
I was so panicked 
I was so upset 
I was very disapointed 
I was just ashamed 
I was really apathetic 
I was super de

#### To get sentences with a NEGATIVE Word and having a First personal Pronoun in its context

In [269]:
"""PRP_sent = list()
neg_word_sent = list()
count  = 0
for i, sent in enumerate(pos_tagged):
    
    tagger = list()
    get_the_tag.flag = 0
    get_the_tag.count_PRP = 0
    get_the_tag.count_neg = 0
    PRP_word = list()
    neg_word = list()
    is_neg_there = False
    for j, word in enumerate(sent):
        tupled = tuple()
        tupled = get_the_tag(word)
        if get_the_tag.count_PRP > 0:
            index = j
            word_before = list()
            word_after = list()
            while index-1 >= 0 and sent[index - 1] is not None and index > j - 3 :
                temp = sent[index - 1]
                tupler = tuple()
                tupler = get_the_tag(temp)
                if tupler[1] == 'NEGATIVE':
                    is_neg_there = True
                    neg_word.append(tupler[0])
                word_before = [tupler[1]] + word_before
                index -= 1
            index = j
            while index + 1 < len(sent) and sent[index + 1] is not None and index < j + 3:
                temp = sent[index + 1]
                tupler = tuple()
                tupler = get_the_tag(temp)
                if tupler[1] == 'NEGATIVE':
                    is_neg_there = True
                    neg_word.append(tupler[0])
                word_after = word_after + [tupler[1]]
                index += 1
            
            if is_neg_there:
                PRP_word = word_before + [word[0]] + word_after
                PRP_sent.append([all_sent_without_BK[i], PRP_word])
                n"""

"PRP_sent = list()\nneg_word_sent = list()\ncount  = 0\nfor i, sent in enumerate(pos_tagged):\n    \n    tagger = list()\n    get_the_tag.flag = 0\n    get_the_tag.count_PRP = 0\n    get_the_tag.count_neg = 0\n    PRP_word = list()\n    neg_word = list()\n    is_neg_there = False\n    for j, word in enumerate(sent):\n        tupled = tuple()\n        tupled = get_the_tag(word)\n        if get_the_tag.count_PRP > 0:\n            index = j\n            word_before = list()\n            word_after = list()\n            while index-1 >= 0 and sent[index - 1] is not None and index > j - 3 :\n                temp = sent[index - 1]\n                tupler = tuple()\n                tupler = get_the_tag(temp)\n                if tupler[1] == 'NEGATIVE':\n                    is_neg_there = True\n                    neg_word.append(tupler[0])\n                word_before = [tupler[1]] + word_before\n                index -= 1\n            index = j\n            while index + 1 < len(sent) and se

In [272]:
len(ourtag)

12782

In [273]:
len(all_sent_without_BK)

12782

In [159]:
#len(PRP_sent)

6013

### Making a dictionary for mapping sentences annotated to comments

In [52]:
csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Top_Level_Comment_Without_Removed.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|', quotechar='"')
i=0
comment_with_id = dict()
for row in reader:
    if i==0: 
        i+=1
        continue
    comment_with_id[row[0]] = row[2]
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Sentence_Comment.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|', quotechar='"')
i=0
sentence_index_to_id = dict()
for row in reader:
    sentence_index_to_id[int(row[2])] = row[1]
csvfile.close()

sent_content_to_sent_index = dict()
for i, sent in enumerate(all_sent_without_BK):
      sent_content_to_sent_index[sent] = i

#### To combine all the context of First personal pronoun and negatives of the same sentence into one list

In [196]:
"""# For finding sentence index if we have only the content of Sentence:
union_PRP = defaultdict(list)
for sent in PRP_sent:
    union_PRP[sentence_with_index[sent[0]]].append(tuple(sent[1]))"""

#### Creating a final dictionary of features with key as the sentence index as appearing in the order of being read and the value being the features of the sentences

In [54]:
finaldict = defaultdict(list)
for i, sent in enumerate(all_sent_without_BK):
    finaldict[i].extend(scored_sentences[i])
    #if union_PRP[i] != []:
    #    finaldict[i].extend(union_PRP[i])

In [55]:
finaldict

defaultdict(list,
            {0: ['Hey everyone,This is just a friendly reminder that discussion of suicide can be really difficult for those who are going  through a tough time and may bring up some scary feelings.',
              0,
              0,
              0,
              1,
              0,
              2,
              4,
              0,
              0,
              2,
              0,
              -4,
              -3],
             1: ['If you are feeling like you are not coping, please know that there are resources out there available to you.',
              0,
              3,
              1,
              0,
              0,
              0,
              0,
              0,
              0,
              5,
              0,
              0,
              4],
             2: ['There are [crisis services worldwide](https://www.iasp.info/resources/Crisis_Centres/) that are trained to provide support.',
              0,
              0,
              0,
           